In [1]:
#import xarray as xr
import numpy as np

In [2]:
import cmomy.accumulator as accumulator
import cmomy.accumulator_central as accumulator_central

In [4]:
from importlib import reload
reload(accumulator)
reload(accumulator_central)

<module 'cmomy.accumulator_central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/accumulator_central.py'>

# Covariance (Higher order)

In [5]:
u = np.random.rand(200)
x = np.random.rand(200)

nu = 5
nx = 10

In [6]:
test = np.zeros((nu+1, nx+1))
dx = x - x.mean()
du = u - u.mean()

for i in range(nu+1):
    for j in range(nx+1):
        if i == 0 and j == 0:
            other = len(x)
        elif i + j == 1:
            other = (u**i * x**j).mean()
        else:
            other = (du**i * dx**j).mean()
        test[i, j] = other

#        np.testing.assert_allclose(out[i,j], other)
#         print(i, j, out[i,j]- other)


In [275]:
accumulator_central.central_comoments(u, x, (2,1)).shape

(3, 2)

In [214]:
reload(accumulator_central)

<module 'cmomy.accumulator_central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/accumulator_central.py'>

In [215]:
datac = np.zeros((nu+1, nx+1))
#datac = np.zeros((10, 2))

In [216]:
%%timeit -n 1 -r 1
datac[...] = 0.0
accumulator_central._push_vals_cov(datac, np.ones_like(u), u, x)

391 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [217]:
np.testing.assert_allclose(datac, test)

In [218]:
reload(accumulator_central)

<module 'cmomy.accumulator_central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/accumulator_central.py'>

In [219]:
s = accumulator_central.StatsAccumCov((nu, nx))

s.push_vals(u, x)

np.testing.assert_allclose(s.data, datac)

In [220]:
%%timeit -n 1 -r 1
s = accumulator_central.StatsAccumCov.from_vals(u, x, moments=(nu,nx))
np.testing.assert_allclose(s.data, datac)

1.15 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [221]:
s = accumulator_central.StatsAccum.from_data(datac)
np.testing.assert_allclose(s.data, datac)

In [222]:
reload(accumulator_central)

<module 'cmomy.accumulator_central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/accumulator_central.py'>

In [223]:
%%timeit -n 1 -r 1
out = accumulator_central.central_comoments(u, x, (nu, nx))
np.testing.assert_allclose(out, s.data)

711 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [224]:
a = np.random.rand(100, 100)
b = np.random.rand(100)

In [225]:

datav = np.zeros((nu+1, nx+1) + a.shape[1:])
datav[...] = 0

accumulator_central._push_vals_cov_vec(datav, np.ones_like(a), 
                                      a, 
                                      np.broadcast_to(b[:,None], a.shape)
                                     )

In [237]:
%%timeit -n 1 -r 1
out2 = accumulator_central.central_comoments(a, b, (nu,nx), out=out)

5.39 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [238]:
np.testing.assert_allclose(out, datav)

In [226]:
%%timeit -n 1 -r 1
out = accumulator_central.central_comoments(a, b, (nu,nx))

np.testing.assert_allclose(datav, out)

5.66 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [227]:
datac[...] = 0.0
accumulator_central._push_vals_cov(datac, np.ones_like(b), a[:,5], b)

In [257]:
reload(accumulator_central)

<module 'cmomy.accumulator_central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/accumulator_central.py'>

In [258]:
s = accumulator_central.StatsAccumCovVec((nu, nx), shape=a.shape[1:])
s.push_vals(a, b, broadcast=True)

np.testing.assert_allclose(s.data, datav)

In [263]:
a = s.reduce(0)

In [264]:
b = accumulator_central.StatsAccumCov.from_datas(s.data, axis=2)

In [269]:
a.data[...,0] - b.data

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [231]:
%%timeit -n 1 -r 1
s = accumulator_central.StatsAccumCovVec.from_vals(a, b, moments=(nu, nx), broadcast=True)
np.testing.assert_allclose(s.data, datav)

43 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [114]:
s0 = s.zeros_like()
s1 = s.zeros_like()

s0.push_vals(a[:20], b[:20], broadcast=True)
s1.push_vals(a[20:], b[20:], broadcast=True)

In [115]:
np.testing.assert_allclose((s0 + s1).data, s.data)
np.testing.assert_allclose((s - s1).data, s0.data)

In [116]:
s0 += s1
np.testing.assert_allclose(s0.data, s.data)

In [117]:
s0 -= s1
np.testing.assert_allclose(s0.data, (s-s1).data)

In [118]:
np.testing.assert_allclose((s0 + s1).data, s.data)

In [119]:
xx = np.random.rand(2, 3, 4, 5)

In [135]:
np.squeeze(np.random.rand(10).mean(0, keepdims=True), 0)

array(0.38953692)

In [129]:
np.expand_dims(np.randomxx.mean(0), 0).shape

(1, 3, 4, 5)

In [132]:
np.squeeze(xx.mean(0,keepdims=True), 1).shape

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [124]:
np.expand_dims(xx, axis=[1,2]).shape

(2, 1, 1, 3, 4, 5)

In [40]:
%%timeit -n 1 -r 1
datac[...] = 0.0
accumulator_central._central_cov_from_vals(datac, np.ones_like(u), u, x)

662 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [41]:
np.testing.assert_allclose(datac, test)

In [505]:
np.testing.assert_allclose(datac, out)

In [478]:
uu = np.random.rand(10000)
xx = np.random.rand(10000)
ww = np.ones_like(uu)

In [487]:
%%timeit -n 1 -r 1
datac[...] = 0.0
accumulator_central._push_vals_cov(datac, ww, uu, xx)

37.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [489]:
datac2 = np.zeros_like(datac)

In [490]:
%%timeit -n 1 -r 1
datac2[...] = 0.0
accumulator_central._central_cov_from_vals(datac2, ww, uu, xx)

7.55 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [492]:
datac - datac2

array([[ 0.00000000e+00,  2.55351296e-15, -2.63677968e-16,
         1.30104261e-18,  6.07153217e-17,  5.96311195e-19,
        -5.07406617e-17,  9.62229428e-19, -6.83047369e-18,
         3.91329222e-19, -1.38235777e-18],
       [ 5.55111512e-16,  8.07730619e-18,  2.16840434e-19,
        -3.37966146e-19, -7.58941521e-19,  3.31189882e-19,
        -3.72270980e-19,  7.00390368e-20, -6.29583708e-20,
         7.51741741e-21, -1.76421081e-20],
       [ 6.93889390e-17,  3.74049750e-18,  8.67361738e-18,
        -1.13841228e-18, -4.29344060e-17,  7.18283939e-19,
        -6.55942314e-18,  4.57397792e-20, -1.11130723e-18,
         1.56701095e-20, -2.71050543e-20],
       [ 9.45966395e-18, -6.23416249e-19,  1.50433051e-18,
        -6.60685699e-20, -2.38863291e-19, -5.17748889e-20,
        -5.92923063e-20,  9.56882533e-21, -1.37907552e-20,
         3.02748104e-21, -1.41571962e-21],
       [ 9.02056208e-17,  4.74338450e-19, -3.90312782e-17,
         4.79420648e-19, -9.54097912e-18,  1.79994501e-19,
  

In [485]:
datac - datac2

array([[-1.00000000e+04, -5.05747358e-01, -8.37505842e-02,
         5.62929357e-04, -1.26190181e-02,  1.83256401e-04,
        -2.26193253e-03,  5.05544108e-05, -4.41249604e-04,
         1.34273015e-05, -9.05231926e-05],
       [-5.00505631e-01, -1.37232781e-04, -2.51927068e-04,
        -7.21836609e-06, -3.29963379e-05, -1.04715539e-06,
        -3.45273455e-06, -4.40483519e-07, -9.22412578e-08,
        -1.97543164e-07,  1.07782413e-07],
       [-8.40306722e-02,  2.49110393e-04, -6.95881824e-03,
         8.04477808e-05, -1.03858353e-03,  2.02527247e-05,
        -1.84820196e-04,  4.99968937e-06, -3.58303935e-05,
         1.24808236e-06, -7.30927596e-06],
       [-2.42309205e-05, -1.03117371e-04, -5.97053884e-05,
        -8.73957472e-06, -9.02403555e-06, -9.05589296e-07,
        -1.19125791e-06, -1.07732175e-07, -1.24322506e-07,
        -1.79621655e-08, -2.37374898e-09],
       [-1.25584733e-02,  4.33878585e-05, -1.03251298e-03,
         1.20670806e-05, -1.53209210e-04,  2.89716592e-06,
  

In [476]:
np.testing.assert_allclose(datac,out)

In [405]:
datac0 = np.zeros_like(datac)
datac1 = np.zeros_like(datac)
datacS = np.zeros_like(datac)

uu = u[:20]
xx = x[:20]
accumulator_central._push_vals_cov(datac0, np.ones_like(uu), uu, xx)

uu = u[20:]
xx = x[20:]
accumulator_central._push_vals_cov(datac1, np.ones_like(uu), uu, xx)

In [411]:
datacS[...] = 0.0
accumulator_central._push_datas_cov(datacS, np.array([datac0, datac1]))

np.testing.assert_allclose(datac, datacS)

In [442]:
reload(accumulator_central)

<module 'python_dropin_files.accumulator_central' from '../python_dropin_files/accumulator_central.py'>

In [443]:

A = np.stack([d[((1,0), (0,1))] for d in (datac0, datac1)], axis=0)
V = np.stack([d[1:, 1:] for d in (datac0, datac1)], axis=0)
W = np.array([d[0,0] for d in [datac0, datac1]])

In [444]:
datacS[...] = 0.0
#accumulator_central._push_stats_cov(datacS, W, A, V)
accumulator_central._push_stat_cov(datacS, W[0], A[0], V[0])

In [445]:
datacS[:,0]

array([2.00000000e+01, 4.94065278e-01, 5.49867192e-06, 2.17657183e-06,
       1.16134927e-06, 5.07345753e-07])

In [446]:
datac0[:,0]

array([2.00000000e+01, 4.94065278e-01, 7.06087533e-02, 1.22993455e-04,
       1.12885602e-02, 1.48249169e-04])

In [379]:
s = accumulator_central.StatsAccum.from_datas(np.array([datac0[:,0], datac1[:,0]]), nmom=5)

In [380]:
s.data

array([2.00000000e+03, 4.95026001e-01, 8.54259965e-02, 3.56203719e-04,
       1.29406646e-02, 1.85527035e-04])

In [337]:
datac[:,0]

array([2.00000000e+03, 4.95026001e-01, 8.54259965e-02, 3.56203719e-04,
       1.29406646e-02, 1.85527035e-04])

In [325]:
datacS[:,0]

array([ 2.00000000e+03,  4.95026001e-01,  8.08198334e-06, -2.81530667e-07,
        1.22397665e-06, -4.23898766e-08])

In [326]:
datac[:,0]

array([2.00000000e+03, 4.95026001e-01, 8.54259965e-02, 3.56203719e-04,
       1.29406646e-02, 1.85527035e-04])

In [319]:
datacS[1,0]

0.49502600082312737

In [316]:
datac[1,0]

0.49502600082312714

In [311]:
A[1]

array([0.49503571, 0.50762568])

In [312]:
datac1[((1,0), (0,1))]

array([0.49503571, 0.50762568])

In [306]:
A[0,1]

0.496796674548943

In [305]:
datacS[0,1]

0.502211174994026

In [304]:
out[0,1]

0.5075173854302096

In [290]:
datacS

array([[ 2.00000000e+00,  5.02211175e-01,  1.01080639e-04,
        -5.37992674e-05,  2.37286854e-05, -1.17291465e-05,
         5.55744710e-06, -2.54059830e-06,  1.22524460e-06,
        -5.50963547e-07,  2.65394434e-07],
       [ 4.94550492e-01, -2.92286418e-03,  1.45465039e-03,
        -8.91362214e-04,  4.67504207e-04, -1.99814663e-04,
         1.08329928e-04, -4.36300306e-05,  2.36578846e-05,
        -9.46885645e-06,  5.08730176e-06],
       [ 7.03409818e-06, -2.17668575e-03,  6.37541129e-03,
        -6.82686408e-04,  9.65852798e-04, -1.52879019e-04,
         1.75683867e-04, -3.21705946e-05,  3.46049442e-05,
        -6.69287383e-06,  7.11349127e-06],
       [ 9.36964878e-07, -7.32745396e-04,  2.51127148e-04,
        -2.12368065e-04,  9.25112337e-05, -4.71873498e-05,
         2.24309233e-05, -1.03576140e-05,  5.01714349e-06,
        -2.26765107e-06,  1.09431671e-06],
       [ 1.19062738e-06, -5.39121310e-04,  1.02141293e-03,
        -1.35066385e-04,  1.60441130e-04, -3.04416662e-05,
  

In [219]:
datac[1,0], out[1,0]

(0.49502600082312714, 0.49502600082312714)

In [220]:
datac[0,1], out[0,1]

(0.5075173854302097, 0.5075173854302096)

In [201]:
np.testing.assert_allclose(datac, out)

In [243]:
datac[...] = 0.0
for uu, xx in zip(u, x):
    accumulator_central._push_stat_cov(datac, 1.0, np.array([uu, xx]), np.zeros((nu-1, nx-1)))


In [249]:
np.testing.assert_allclose(out,datac)

In [247]:
np.testing.assert_allclose(out[:4,:4],datac[:4,:4])

In [224]:
out[2,1]

0.00018849835769094766

In [225]:
datac[2,1]

0.0001884983576909486

In [221]:
out[2,0]

0.08542599652601496

In [223]:
datac[2, 0]

0.08542599652601496

In [165]:
datac = np.zeros((nu+1, nx+1))
for (uu, vv) in zip(u, x):
    #accumulator_central._get_moments(datac, 1.0, uu, vv)
    accumulator_central._push_val_cov(datac, 1.0, uu, vv)


In [124]:
datac[0,1]

0.5075173854302097

In [125]:
datac[1,0]

0.49502600082312714

In [126]:
datac[1,1]

0.0044924415019052906

In [127]:
out[1,1], datac[1,1]

(0.004492441501905286, 0.0044924415019052906)

In [114]:
out[1,0]

0.49502600082312714

In [128]:
np.testing.assert_allclose(out,datac)

In [52]:
a0 = 3
a1 = 5

In [67]:
i = 0
for b0 in range(0, a0+1):
    c0 = a0 - b0
    for b1 in range(0, a1+1):
        c1 = a1 - b1
        
        if c0 + c1 == 1:
            #print('one', b0, b1)
            pass
            
        elif c0 + c1 == 0:
            print('zero', b0, b1)
            #pass
            
            
        else:
            print('other', i, b0, b1, c0, c1)
            i += 1

other 0 0 0 3 5
other 1 0 1 3 4
other 2 0 2 3 3
other 3 0 3 3 2
other 4 0 4 3 1
other 5 0 5 3 0
other 6 1 0 2 5
other 7 1 1 2 4
other 8 1 2 2 3
other 9 1 3 2 2
other 10 1 4 2 1
other 11 1 5 2 0
other 12 2 0 1 5
other 13 2 1 1 4
other 14 2 2 1 3
other 15 2 3 1 2
other 16 2 4 1 1
other 17 3 0 0 5
other 18 3 1 0 4
other 19 3 2 0 3
other 20 3 3 0 2
zero 3 5


In [99]:
a1 + 1

6

In [94]:
i = 0
for b0 in range(0, a0+1):
    c0 = a0 - b0
    
    b1_max = min(a1 + c0, a1+1)
    print(b1_max)
    for b1 in range(0, b1_max):
        c1 = a1 - b1
        
        if c0 + c1 == 1:
            print('extra', b0, b1, c0, c1)
            pass
            
        elif c0 + c1 == 0:
            print('zero', b0, b1, c0, c1)
            pass
            
            
        else:
            print('other', i, b0, b1, c0, c1)
            i += 1

6
other 0 0 0 3 5
other 1 0 1 3 4
other 2 0 2 3 3
other 3 0 3 3 2
other 4 0 4 3 1
other 5 0 5 3 0
6
other 6 1 0 2 5
other 7 1 1 2 4
other 8 1 2 2 3
other 9 1 3 2 2
other 10 1 4 2 1
other 11 1 5 2 0
6
other 12 2 0 1 5
other 13 2 1 1 4
other 14 2 2 1 3
other 15 2 3 1 2
other 16 2 4 1 1
extra 2 5 1 0
5
other 17 3 0 0 5
other 18 3 1 0 4
other 19 3 2 0 3
other 20 3 3 0 2
extra 3 4 0 1


In [19]:
x0, x1 = np.random.rand(2)
alpha = np.random.rand()

In [26]:
list(range(0, 0+1))

[0]

In [28]:
list(range(0, -1))

[]

In [70]:
alpha_0 = 1.0
minus_0 = 1.0
for b0 in range(0, a0+1):
    c0 = a0 - b0
    
    alpha_bb = alpha_0
    minus_bb = minus_0
    for b1 in range(0, a1+1):
        c1 = a1 - b1
        
        

        alpha_bb *= alpha
        minus_bb *= -1
        
    alpha_0 *= alpha
    minus_0 *= -1
        
    

In [77]:
alpha_0, alpha ** (a0+1)

(0.08609356345020769, 0.08609356345020767)

In [85]:
alpha_bb, alpha**(a0+a1+1)

(0.004014986031001154, 0.004014986031001153)

In [21]:
alpha_0, alpha ** (a0-1)

(0.29341704696593157, 0.29341704696593157)

In [22]:
alpha ** (a0 + a1 - 3), alpha_bb

(0.046635147508131955, 0.04663514750813196)

In [24]:
(-1)**(a0 + a1 - 3), minus_bb

(-1, -1.0)

In [969]:
alpha ** (a0-1 + a1)

0.3405555290380999

In [963]:
alpha ** (a0 + a1 - 4)

0.58357135728041

In [952]:
alpha**(a0 + a1)

0.2845897526522902

In [953]:
alpha_bb * alpha_0 * alpha ** 2

0.23782120802282072

In [918]:
range(0, 1)

range(0, 1)

In [919]:
alpha

0.8539514687136144

In [920]:
alpha_bb

0.8539514687136144

In [31]:
np.testing.assert_allclose(out,datac)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 66 / 66 (100%)
Max absolute difference: 1999.
Max relative difference: 1999.
 x: array([[ 2.000000e+03,  5.075174e-01,  8.409365e-02, -8.856088e-04,
         1.272233e-02, -2.829138e-04,  2.294010e-03, -7.759181e-05,
         4.507539e-04, -2.041362e-05,  9.321942e-05],...
 y: array([[1.      , 0.141149, 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.597173, 0.      , 0.      , 0.      , 0.      , 0.      ,...

In [862]:
out[1,2]

0.00023076759905562492

In [863]:
datac[1,2]

0.00023076759905562444

In [845]:
datac[2,1]

0.0033627321356257604

In [837]:
out[2,0] - datac[2,0]

0.0

In [834]:
datac[2,0]

0.08446688770432603

In [824]:
out[2,1]

0.00012665210827644818

In [825]:
datac[2,1]

0.0033627321356257604

In [801]:
out[2,0]

0.08446688770432603

In [802]:
datac[2,0]

6.817508765979018e-07

In [796]:
out[2,0] - datac[2,0]

0.08446620595344943

In [794]:
out[0,1] - datac[0,1]

0.0

In [783]:
datac[2,2] - out[2,2]

-1.726181513778298e-05

In [773]:
out[1,0]

0.5030104742189434

In [774]:
datac[1,0]

0.5030104742189434

In [775]:
datac[0,1]

0.5044784891474983

In [776]:
datac[2,2]

0.007103773039984232

In [777]:
out[2,2]

0.007121034855122015

0.5044784891474983

In [751]:
out.shape

(6, 11)

In [743]:
from importlib import reload
reload(accumulator_central)

s = nu+1
data = np.zeros(s)
accumulator_central._push_vals(data, np.ones_like(u), u)
# for uu in u:
#     accumulator_central._push_val_nmom(data, 1.0, uu)

In [719]:
reload(accumulator_central)

<module 'accumulator_central' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator_central.py'>

In [720]:
np.testing.assert_allclose(data, test[:, 0])

In [721]:
data0 = np.zeros(s)
for uu in u[:4]:
    accumulator_central._push_val(data0, 1.0, uu)
    
data1 = np.zeros(s)
for uu in u[4:10]:
    accumulator_central._push_val(data1, 1.0, uu)
    
data2 = np.zeros(s)
for uu in u[10:]:
    accumulator_central._push_val(data2, 1.0, uu)
    

In [722]:
datac = np.zeros(s)

for d in [data0, data1, data2]:
    accumulator_central._push_stat(datac, w=d[0], a=d[1], v=d[2:])


In [723]:
np.testing.assert_allclose(datac, test[:, 0])

In [724]:
reload(accumulator_central)

<module 'accumulator_central' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator_central.py'>

In [725]:
s = accumulator_central.StatsAccum(nmom=nu)

In [726]:
reload(accumulator_central)

<module 'accumulator_central' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator_central.py'>

In [727]:
%%timeit -n 1 -r 1
s = accumulator_central.StatsAccum.from_vals(u, nmom=nu)
np.testing.assert_allclose(s.data, data)

222 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [728]:
%%timeit -n 1 -r 1
s = accumulator_central.StatsAccum.from_datas(np.stack([data0, data1, data2]), nmom=nu)
np.testing.assert_allclose(s.data, data)

262 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [729]:
s = accumulator_central.StatsAccum.from_datas(np.stack([data0, data1, data2]), nmom=nu)


In [730]:
# adding

In [731]:
reload(accumulator_central)

<module 'accumulator_central' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator_central.py'>

In [732]:
a = len(u) // 3
b = 2 * a

In [704]:
su = accumulator_central.StatsAccum.from_vals(u, nmom=nu)

In [705]:
s0 = accumulator_central.StatsAccum.from_vals(u[:a], nmom=nu)
s1 = accumulator_central.StatsAccum.from_vals(u[a:b], nmom=nu)
s2 = accumulator_central.StatsAccum.from_vals(u[b:], nmom=nu)

In [706]:
new = s0.copy()

In [707]:
new.push_stat(w=s1.weight(), a=s1.mean(), v=s1.cmom())

In [708]:
(s0 + s1 + s2).data - data

array([ 2.00000000e+03,  5.03010474e-01,  8.44668877e-02, -4.41663483e-04,
        1.26805092e-02, -1.52001435e-04])

In [709]:
(su -s1 - s2).data - s0.data


array([ 0.00000000e+00, -2.22044605e-16,  2.49800181e-16, -7.04731412e-18,
       -1.23165367e-16,  3.87602277e-18])

In [710]:
# vector
su = accumulator_central.StatsAccum.from_vals(u, nmom=nu)
sx = accumulator_central.StatsAccum.from_vals(x, nmom=nu)

In [711]:
y = np.stack([u, x], -1)

In [712]:
sv = accumulator_central.StatsAccumVec.from_vals(np.stack([u, x], -1), nmom=nu)

In [523]:
sv.data[:,0] - su.data

array([0., 0., 0., 0., 0., 0.])

In [524]:
sv.data[:, 1] - sx.data

array([0., 0., 0., 0., 0., 0.])

In [525]:
import xarray as xr

In [526]:
a = xr.DataArray(np.random.rand(5,4), dims=['b','a'])
b = xr.DataArray(np.random.rand(4,5,6), dims=['a','b','c'])

In [562]:
# playing with sizes

In [48]:

reload(accumulator_central)
reload(accumulator)
import numpy as np

In [121]:
nmom = 10
nv = 500

In [122]:
v = np.random.rand(10000, nv)
wv = np.ones_like(v)

In [123]:
y = np.array(v[:,0], order='c')
wy = np.ones_like(y)

In [124]:
data = np.zeros(nmom+1)
datav = np.zeros((nmom+1, nv))

In [125]:
datavc = np.zeros_like(datav)

In [126]:
reload(accumulator_central)

<module 'cmomy.accumulator_central' from '/Users/wpk/Documents/python/projects/cmomy/examples/cmomy/accumulator_central.py'>

In [127]:
%%timeit -n 1 -r 1
datavc[...] = 0
accumulator_central._central_from_vals_vec(datavc, wv, v)

1.12 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [129]:
%%timeit -n 1 -r 1
datav[...] = 0.0
accumulator_central._push_vals_vec(datav, wv, v)

478 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [112]:
%%timeit -n 1 -r 1
datav[...] = 0.0
accumulator_central._push_vals_vec(datav, wv, v)

56 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [130]:
np.testing.assert_allclose(datavc, datav)

In [62]:
datav

array([[ 1.00000000e+03,  1.00000000e+03,  1.00000000e+03, ...,
         1.00000000e+03,  1.00000000e+03,  1.00000000e+03],
       [ 5.00270972e-01,  4.96942317e-01,  5.14454403e-01, ...,
         5.02562179e-01,  4.92502398e-01,  5.07601508e-01],
       [ 8.41940412e-02,  8.43581579e-02,  8.17545222e-02, ...,
         7.91594938e-02,  8.54410484e-02,  8.22942800e-02],
       [-6.43473391e-05, -7.21287231e-06, -1.36301494e-03, ...,
        -7.36768704e-04,  1.23353312e-03, -1.34180469e-03],
       [ 1.26651253e-02,  1.25223108e-02,  1.20719801e-02, ...,
         1.16175679e-02,  1.31191423e-02,  1.22497030e-02],
       [-3.77844935e-05, -3.34987237e-05, -5.47515372e-04, ...,
        -2.66155829e-04,  4.35729829e-04, -3.81241119e-04]])

In [63]:
datav2 = np.zeros((nv, nmom+1))

In [64]:
%%timeit -n 1 -r 1
datav2[...]= 0.0
accumulator._push_vals_vec(datav2, wv, v)

256 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [65]:
np.testing.assert_allclose(datav[:3,...], datav2.T[:3, ...])

In [66]:
%%timeit -n 1 -r 1
datav2[...] = 0.0
accumulator_central._push_vals_vec(datav, wv, v)

187 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [73]:
%%timeit -n 10 -r 3
data[...] = 0.0
accumulator_central._push_vals(data, wy, y)

26.9 µs ± 726 ns per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [70]:
data2 = np.zeros_like(data)

In [74]:
%%timeit -n 10 -r 3
data2[...] = 0.0
accumulator_central._central_from_vals(data2, wy, y)

177 µs ± 31.6 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [29]:
np.testing.assert_allclose(data, data2)

In [30]:
np.testing.assert_allclose(data, accumulator_central.StatsAccum.from_vals(y, nmom=nmom).data)

In [46]:
reload(accumulator_central)

<module 'accumulator_central' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator_central.py'>

In [48]:
%%timeit -n 1 -r 1
datav[...] = 0.0
accumulator_central._push_vals_vec(datav, wv, v)

7.94 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [65]:
datav.shape

(11, 100)

In [63]:
v[0,:].shape

(100,)

In [69]:
datav[0,:]

array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.])

In [50]:
accumulator_central.StatsAccumVec.from_vals(v, nmom=nmom).data

array([[ 1.00000000e+03,  1.00000000e+03,  1.00000000e+03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.84468788e-01,  5.09991682e-01,  5.17525979e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.66453600e-02,  8.42371702e-02,  8.01758415e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 4.95210304e-04,  4.49922560e-04,  4.11519044e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.30306436e-05, -3.03834063e-05, -4.67353197e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.04660234e-04,  9.29903994e-05,  8.49490620e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [ ]:
acc

In [26]:
np.testing.assert_allclose(datav, accumulator_central.StatsAccumVec.from_vals(v, nmom=nmom).data)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(shapes (11, 100), (100, 11) mismatch)
 x: array([[ 1.000000e+03,  1.000000e+03,  1.000000e+03, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       [ 4.844688e-01,  5.099917e-01,  5.175260e-01, ...,  0.000000e+00,...
 y: array([[1000., 1000., 1000., ..., 1000., 1000., 1000.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],...

In [1010]:
datav2 = np.zeros((nmom+1, nv))

In [1011]:
%%timeit -n 1 -r 1
datav2[...] = 0.0
accumulator_central._push_vals_vec_nmom2(datav2, wv, v)

498 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [1012]:
%%timeit -n 1 -r 1
datav2[...] = 0.0
accumulator_central._vec_push_vals_nmom(datav2, wv, v)


356 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [973]:
datav2.T - datav

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [913]:
datav2.T

array([[1.00000000e+03, 5.00754809e-01, 8.48214609e-02],
       [1.00000000e+03, 4.96995540e-01, 8.23855192e-02],
       [1.00000000e+03, 4.96221002e-01, 8.36091617e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+03, 5.02764973e-01, 8.16607873e-02],
       [1.00000000e+03, 5.01010179e-01, 8.43597307e-02],
       [1.00000000e+03, 4.98194914e-01, 8.53425835e-02],
       [1.00000000e+03, 4.98087142e-01, 8.34877660e-02],
       [1.00000000e+03, 5.03279532e-01, 8.14883356e-02],
       [1.00000000e+03, 4.92091896e-01, 8.33388170e-02],
       [1.00000000e+03, 4.99164371e-01, 8.30120104e-02]])

In [890]:
wys = wy.reshape(-1, 1)
ys = y.reshape(-1, 1)

In [892]:
%%timeit -n 10 -r 3
data2[...] = 0.0
accumulator_central._vec_push_vals_nmom(data2, wys, ys)

1.78 ms ± 23.9 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [880]:
data

array([1.00000000e+03, 4.82649028e-01, 8.48409676e-02])

In [881]:
data

array([[1.00000000e+03],
       [4.82649028e-01],
       [8.48409676e-02]])

In [855]:
datav2[...] = 0.0

In [858]:
%%timeit -n 10 -r 3
datav2[...] = 0.0
accumulator_central._vec_push_vals_nmom(datav2, wv, v)

1.24 ms ± 9.9 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [833]:
datav - datav2.T

array([[1.00000000e+03, 4.88233093e-01, 8.16752775e-02],
       [1.00000000e+03, 4.91592016e-01, 8.12709749e-02],
       [1.00000000e+03, 5.00183774e-01, 8.63580362e-02],
       [1.00000000e+03, 5.09298181e-01, 8.03620055e-02],
       [1.00000000e+03, 4.97363010e-01, 8.42037389e-02],
       [1.00000000e+03, 5.10687036e-01, 8.62974788e-02],
       [1.00000000e+03, 5.01327530e-01, 8.26025968e-02],
       [1.00000000e+03, 4.96930042e-01, 8.37995733e-02],
       [1.00000000e+03, 4.87614174e-01, 8.49292351e-02],
       [1.00000000e+03, 4.92219122e-01, 8.42484274e-02]])

In [768]:
datav2.T

array([[1.00000000e+03, 4.88233093e-01, 8.16752775e-02],
       [1.00000000e+03, 4.91592016e-01, 8.12709749e-02],
       [1.00000000e+03, 5.00183774e-01, 8.63580362e-02],
       [1.00000000e+03, 5.09298181e-01, 8.03620055e-02],
       [1.00000000e+03, 4.97363010e-01, 8.42037389e-02],
       [1.00000000e+03, 5.10687036e-01, 8.62974788e-02],
       [1.00000000e+03, 5.01327530e-01, 8.26025968e-02],
       [1.00000000e+03, 4.96930042e-01, 8.37995733e-02],
       [1.00000000e+03, 4.87614174e-01, 8.49292351e-02],
       [1.00000000e+03, 4.92219122e-01, 8.42484274e-02]])

In [ ]:
accum

In [666]:
data2 = np.zeros((nu+1, 1))

In [693]:
reload(accumulator_central)

<module 'python_dropin_files.accumulator_central' from '../python_dropin_files/accumulator_central.py'>

In [694]:
#%%timeit -n 1 -r 1 
data2[...] = 0.0
# for yy in y:
#     accumulator_central._push_val_gen(data2, np.array([1.0]), np.array([yy]))
accumulator_central._push_vals_gen(data2, wy[:,None], y[:,None])

LoweringError: Failed in nopython mode pipeline (step: nopython mode backend)
[1m[1m
[1m
File "accumulator_central.py", line 78:[0m
[1mdef _push_vals_nmom(data, W, X, bfac):
    <source elided>

[1m@myjit
[0m[1m^[0m[0m
[0m
[0m[1m[1] During: lowering "$202binary_add.16.6153 = arrayexpr(expr=(<built-in function add>, [(<built-in function mul>, [(<built-in function mul>, [Var(_178binary_power_4_6141, accumulator_central.py:77), (<built-in function pow>, [Var(alpha_6079, accumulator_central.py:59), Var(b.6128, accumulator_central.py:71)])]), Var(one_alpha.6083, accumulator_central.py:60)]), (<built-in function mul>, [(<built-in function pow>, [Var(one_alpha.6083, accumulator_central.py:60), Var(b.6128, accumulator_central.py:71)]), Var(alpha.6079, accumulator_central.py:59)])]), ty=array(float64, 1d, C))" at ../python_dropin_files/accumulator_central.py (78)[0m

In [689]:
wy[:,None][0]

array([1.])

In [690]:
accumulator_central._push_val_gen(data2, wy[:,None][0], y[:,None][0])

In [669]:
data3 = np.zeros(3)

In [670]:
%%timeit -n 1 -r 1
data3[...] = 0.0
accumulator._push_vals(data3, wy, y)

23.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [671]:
data3 - data

array([0., 0., 0.])

In [672]:
data3v = np.zeros((3, 10))

In [686]:
%%timeit -n 1 -r 1
data3v[...] = 0.0
accumulator._push_vals(data3v, wy, v)

1.25 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [677]:
datav.shape

(10, 3)

In [683]:
data3v.T

array([[1.00000000e+03, 4.93164127e-01, 8.57376084e-02],
       [1.00000000e+03, 4.94909316e-01, 8.26797779e-02],
       [1.00000000e+03, 4.92329586e-01, 8.13586849e-02],
       [1.00000000e+03, 4.92012503e-01, 8.33420792e-02],
       [1.00000000e+03, 4.95635218e-01, 8.30641291e-02],
       [1.00000000e+03, 4.95981720e-01, 8.14779074e-02],
       [1.00000000e+03, 4.82537471e-01, 8.67320718e-02],
       [1.00000000e+03, 5.02259324e-01, 8.18378637e-02],
       [1.00000000e+03, 4.88356759e-01, 8.42744313e-02],
       [1.00000000e+03, 5.04622471e-01, 8.67121726e-02]])

In [639]:
data3

array([1.00000000e+03, 4.93164127e-01, 8.57376084e-02])

In [640]:
data

array([1.00000000e+03, 4.93164127e-01, 8.57376084e-02, 1.09946962e-03,
       1.30475164e-02, 3.75993519e-04])

In [ ]:
from numba import guvectorize, float32, float64

@guvectorize([(float64, float64, float64[:])])

In [ ]:
for 

In [609]:
wy[:,None][0,...]

array([1.])

In [592]:
data2[:,0] - data

array([ 0.00000000e+00,  0.00000000e+00,  8.56934143e-02, -1.30104261e-18,
        1.73472348e-18,  2.16840434e-19])

In [577]:
wy[:,None][0]

array([1.])

In [572]:
data2

array([[1.00000000e+03],
       [4.93164127e-01],
       [8.57376084e-02],
       [1.09946962e-03],
       [1.30475164e-02],
       [3.75993519e-04]])

In [573]:
data

array([1.00000000e+03, 4.93164127e-01, 4.41940640e-05, 1.09946962e-03,
       1.30475164e-02, 3.75993519e-04])

In [556]:
tmp, a = accumulator_central._push_val_gen(data2, 1.0, np.array(yy))


In [561]:
data[a, ...] = tmp

In [ ]:
#alternative
datav2 = np.zeros((nu+1, 10))

@accumulator_central.myjit
def _push_gen()

In [423]:
accumulator_central.StatsAccumVec.from_vals(v, nmom=nu).data - datav

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])